In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
from torchvision import models, transforms


import os
root_dir = "C:/Users/faruk/computer_vision_project/poster_images"
os.path.isfile(os.path.join(root_dir, os.path.basename(df["image_path"][2000])))
print(os.path.join(root_dir, os.path.basename(df["image_path"][2000])))



# -------------------------------------------------------------
# 1.  Model definition (identical to training time)
# -------------------------------------------------------------
class PosterResNetRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(weights=None)      # structure only
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, 1)  # head

    def forward(self, x):
        return self.backbone(x)

# -------------------------------------------------------------
# 2.  Pre‑processing transform (same stats as training)
# -------------------------------------------------------------
infer_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225]),
])

# -------------------------------------------------------------
# 3.  Inference function
# -------------------------------------------------------------
def predict_imdb(poster_path: str,
                 weight_path: str = r"C:\Users\faruk\Downloads\poster_resnet18_final.pth",
                 device: str | torch.device = "cpu") -> float:
    """
    Args
    ----
    poster_path : str
        Path to a single movie‑poster image (JPEG/PNG).
    weight_path : str
        Path to the .pth file containing trained weights.
    device      : "cpu" or "cuda".
    
    Returns
    -------
    float  – predicted IMDb score (0‑10 scale).
    """
    device = torch.device(device)
    
    # 1. Build model & load weights
    model = PosterResNetRegressor().to(device)
    state_dict = torch.load(weight_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()
    
    # 2. Image → tensor
    img = Image.open(poster_path).convert("RGB")
    x   = infer_transform(img).unsqueeze(0).to(device)     # shape (1,3,224,224)
    
    # 3. Forward pass
    with torch.no_grad():
        pred = model(x).cpu().squeeze().item()             # scalar float
    
    return pred

# -------------------------------------------------------------
# 4.  Example usage
# -------------------------------------------------------------
if __name__ == "__main__":
    test_img = r"C:\Users\faruk\Downloads\b.jpg"
    score = predict_imdb(test_img)
    print(f"Predicted IMDb score: {score:.2f}")
